<a href="https://colab.research.google.com/github/vaibhavsadgir50/Projects/blob/main/Age_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub
! pip install -q kaggle
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf


from sklearn.metrics import r2_score
import kagglehub
from google.colab import userdata
import os



from google.colab import files

files.upload()

# Download latest version
#path = kagglehub.dataset_download("mariafrenti/age-prediction")

#print("Path to dataset files:", path)

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaibhavchandgir7218","key":"7ea8567d9d83102ac25638bc5cb24269"}'}

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle  download -c 'mariafrenti/age-prediction'
! mkdir age-prediction
! unzip age-prediction.zip -d age-prediction

Streaming output truncated to the last 5000 lines.
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/78007.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/78332.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/78495.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/78603.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/78950.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/79193.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/79502.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/79589.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/79660.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/79733.jpg  
  inflating: age-prediction/age_prediction_up/age_prediction/train/070/79774.jpg  
  inflating: age-prediction/age_pred

In [ ]:
image_dir = Path('../content/age-prediction/20-50/20-50')

In [ ]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.jpg')), name='Filepath').astype(str)
ages = pd.Series(filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1]), name='Age').astype(np.integer)

images = pd.concat([filepaths, ages], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/common.py:1645: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  npdtype = np.dtype(dtype)


In [ ]:
images

,Filepath,Age
0,../content/age-prediction/20-50/20-50/train/25...,25
1,../content/age-prediction/20-50/20-50/train/41...,41
2,../content/age-prediction/20-50/20-50/train/21...,21
3,../content/age-prediction/20-50/20-50/train/38...,38
4,../content/age-prediction/20-50/20-50/train/40...,40
...,...,...
40435,../content/age-prediction/20-50/20-50/train/37...,37
40436,../content/age-prediction/20-50/20-50/train/35...,35
40437,../content/age-prediction/20-50/20-50/train/34...,34
40438,../content/age-prediction/20-50/20-50/train/23...,23


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_df = images.sample(500, random_state=2).reset_index(drop=True)

train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_df

,Filepath,Age
13,../content/age-prediction/20-50/20-50/train/31...,31
61,../content/age-prediction/20-50/20-50/train/31...,31
298,../content/age-prediction/20-50/20-50/train/31...,31
39,../content/age-prediction/20-50/20-50/train/44...,44
360,../content/age-prediction/20-50/20-50/train/47...,47
...,...,...
255,../content/age-prediction/20-50/20-50/train/22...,22
72,../content/age-prediction/20-50/20-50/test/24/...,24
396,../content/age-prediction/20-50/20-50/test/37/...,37
235,../content/age-prediction/20-50/20-50/train/25...,25


In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 280 validated image filenames.
Found 70 validated image filenames.
Found 150 validated image filenames.


In [ ]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 313ms/step - loss: 1248.0044 - val_loss: 1166.6234
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 295ms/step - loss: 1256.1421 - val_loss: 1027.1519
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 479ms/step - loss: 1034.1290 - val_loss: 674.4463
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 292ms/step - loss: 617.1365 - val_loss: 177.4399
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 276ms/step - loss: 205.1534 - val_loss: 287.1310
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 278ms/step - loss: 186.2916 - val_loss: 140.4734
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 397ms/step - loss: 157.8575 - val_loss: 139.0791
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 281ms/step - loss: 127.9269 - val_loss: 154.4651
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 292ms/step - loss: 145.1796 - val_loss: 147.7633
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 287ms/step - loss: 134.3341 - val_loss: 137.8830
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 280ms/step - loss: 127.6123 - val_loss: 145.5884
Epoch 12/100
9/9 ━━━━━━━━━

In [ ]:
from sklearn.metrics import accuracy_score

predicted_ages = np.squeeze(model.predict(test_images))
true_ages = test_images.labels

accuracy_score = 1 - np.mean(np.abs(predicted_ages - true_ages) / true_ages)
print("Test Accuracy Score: {:.5f}".format(accuracy_score))

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_ages, predicted_ages)
print("Test R^2 Score: {:.5f}".format(r2))

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 366ms/step
Test Accuracy Score: 0.74898
     Test RMSE: 9.39471
Test R^2 Score: -0.00612


In [ ]:
null_rmse = np.sqrt(np.sum((true_ages - np.mean(true_ages))**2) / len(true_ages))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

Null/Baseline Model Test RMSE: 9.36611
